Image Preprocessing

In [7]:
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm

# Define Paths
input_images_dir = r"D:\sonar seg\dataset\Images"  # Change this to your sonar images folder
output_images_dir = r"D:\sonar seg\preprocessed\images"

# Create output directory if it doesn't exist
os.makedirs(output_images_dir, exist_ok=True)

# Define Augmentations (for better segmentation performance)
augmentations = A.Compose([
    A.Resize(640, 640),  # Resize to YOLOv8 standard input size
    A.HorizontalFlip(p=0.5),  # Flip images 50% of the time
    A.Rotate(limit=15, p=0.5),  # Random small rotations
    A.RandomBrightnessContrast(p=0.3),  # Adjust brightness/contrast
    A.GaussianBlur(p=0.2)  # Slight blurring to reduce noise
])

# Function to preprocess sonar images
def preprocess_sonar_image(image_path, output_path):
    try:
        # Load Image
        image = cv2.imread(image_path)

        if image is None:
            print(f"❌ Error loading image: {image_path}")
            return

        # Convert to grayscale (optional for sonar images)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        image = clahe.apply(image)

        # Normalize image (bring pixel values between 0-255)
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)

        # Convert back to 3-channel image (needed for YOLO training)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        # Apply augmentations
        augmented = augmentations(image=image)
        image = augmented["image"]

        # Save processed image
        cv2.imwrite(output_path, image)

    except Exception as e:
        print(f"⚠️ Error processing {image_path}: {e}")

# Process all images
image_files = [f for f in os.listdir(input_images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

for image_name in tqdm(image_files, desc="Processing Images"):
    input_path = os.path.join(input_images_dir, image_name)
    output_path = os.path.join(output_images_dir, image_name)
    preprocess_sonar_image(input_path, output_path)

print("✅ Image Preprocessing Complete! Images saved in:", output_images_dir)


C:\Users\VARSHITH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing Images: 100%|██████████| 1868/1868 [00:39<00:00, 47.87it/s]

✅ Image Preprocessing Complete! Images saved in: D:\sonar seg\preprocessed\images


Label Conversion

In [2]:
import os
import xml.etree.ElementTree as ET
import glob

# Paths
xml_folder = r"D:\sonar seg\dataset\BoxAnnotations"  # Folder with XML files
output_yolo_folder = r"D:\sonar seg\preprocessed\labels"  # Output folder for YOLO labels
os.makedirs(output_yolo_folder, exist_ok=True)

# Class Mapping
class_mapping = {
    "Bottle": 0,
    "Can": 1,
    "Chain": 2,
    "Drink-carton": 3,
    "Hook": 4,
    "Propeller": 5,
    "Shampoo-bottle": 6,
    "Standing-bottle": 7,
    "Tire": 8,
    "Valve": 9,
    "Wall": 10
}

# Convert XML to YOLOv8 segmentation format
def convert_xml_to_yolo(xml_file, output_file, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    with open(output_file, "w") as f:
        object_found = False

        for obj in root.findall("object"):
            class_name = obj.find("name").text.strip()
            if class_name not in class_mapping:
                print(f" Unknown class '{class_name}' in {xml_file}, skipping...")
                continue

            class_id = class_mapping[class_name]

            # Extract COCO-style bounding box (x, y, w, h)
            bndbox = obj.find("bndbox")
            if bndbox is not None:
                try:
                    x = float(bndbox.find("x").text)
                    y = float(bndbox.find("y").text)
                    w = float(bndbox.find("w").text)
                    h = float(bndbox.find("h").text)

                    # Convert (x, y, w, h) → (xmin, ymin, xmax, ymax)
                    xmin = x / img_width
                    ymin = y / img_height
                    xmax = (x + w) / img_width
                    ymax = (y + h) / img_height

                    # YOLOv8 expects segmentation points (rectangle for now)
                    polygon = [xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax]
                    f.write(f"{class_id} " + " ".join(map(str, polygon)) + "\n")
                    object_found = True

                except (AttributeError, ValueError):
                    print(f" Missing or invalid bounding box in: {xml_file}")
                    continue

        if not object_found:
            print(f" No valid objects found in {xml_file}, skipping...")

# Process all XML files
xml_files = glob.glob(os.path.join(xml_folder, "*.xml"))
for xml_file in xml_files:
    filename = os.path.basename(xml_file).replace(".xml", ".txt")
    output_file = os.path.join(output_yolo_folder, filename)

    tree = ET.parse(xml_file)
    root = tree.getroot()

    try:
        img_width = int(root.find("size/width").text)
        img_height = int(root.find("size/height").text)
    except (AttributeError, ValueError):
        print(f" ERROR: Missing or invalid <size> tag in {xml_file}")
        continue

    convert_xml_to_yolo(xml_file, output_file, img_width, img_height)

print(" XML to YOLOv8 segmentation format conversion complete!")


 XML to YOLOv8 segmentation format conversion complete!


Class Extraction

In [3]:
import os
import xml.etree.ElementTree as ET
import glob

# Paths
xml_folder = r"D:\sonar seg\dataset\BoxAnnotations"  # Folder with XML files
output_classes_file = r"D:\sonar seg\preprocessed\classes.txt"  # Output file for class names

# Extract unique class names
unique_classes = set()

# Process all XML files
xml_files = glob.glob(os.path.join(xml_folder, "*.xml"))
for xml_file in xml_files:
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for obj in root.findall("object"):
        class_name = obj.find("name").text.strip()
        unique_classes.add(class_name)

# Sort classes alphabetically and assign class IDs
sorted_classes = sorted(unique_classes)
class_mapping = {class_name: idx for idx, class_name in enumerate(sorted_classes)}

# Save class names with assigned IDs
with open(output_classes_file, "w") as f:
    for class_name, class_id in class_mapping.items():
        f.write(f"{class_id} {class_name}\n")

# Print the mapping
print(f"✅ Extracted {len(sorted_classes)} unique classes. Saved to: {output_classes_file}")
print("Class Mapping:")
for class_name, class_id in class_mapping.items():
    print(f"{class_id}: {class_name}")


✅ Extracted 11 unique classes. Saved to: D:\sonar seg\preprocessed\classes.txt
Class Mapping:
0: Bottle
1: Can
2: Chain
3: Drink-carton
4: Hook
5: Propeller
6: Shampoo-bottle
7: Standing-bottle
8: Tire
9: Valve
10: Wall


Test Train Val Split

In [20]:
print(f"Found {len(os.listdir(labels_dir))} label files in {labels_dir}")

Found 1868 label files in D:\sonar seg\preprocessed\labels


In [5]:
import os
import shutil
import random
from collections import defaultdict

# ✅ Define dataset directories (SET YOUR PATHS HERE)
images_dir = r"D:\sonar seg\dataset\Images"
labels_dir = r"D:\sonar seg\preprocessed\labels"
output_base_dir = r"D:\sonar seg\ttv"

# Output directories
output_dirs = {
    "train": {"images": os.path.join(output_base_dir, "train", "images"),
              "labels": os.path.join(output_base_dir, "train", "labels")},
    "val": {"images": os.path.join(output_base_dir, "val", "images"),
            "labels": os.path.join(output_base_dir, "val", "labels")},
    "test": {"images": os.path.join(output_base_dir, "test", "images"),
             "labels": os.path.join(output_base_dir, "test", "labels")}
}

# ✅ Create directories if they don't exist
for split in output_dirs.values():
    for path in split.values():
        os.makedirs(path, exist_ok=True)

# ✅ Organize files by class
class_files = defaultdict(list)
found_labels = os.listdir(labels_dir)

if not found_labels:
    print("❌ ERROR: No label files found in the labels directory!")
    exit()

print(f"Found {len(found_labels)} label files in {labels_dir}")

for label_file in found_labels:
    if label_file.endswith(".txt"):
        with open(os.path.join(labels_dir, label_file), "r") as f:
            lines = f.readlines()
            if lines:
                class_id = int(lines[0].split()[0])  # Get the first class ID in the file
                class_files[class_id].append(label_file)
            else:
                print(f"⚠️ Empty label file: {label_file}")

# ✅ Split dataset while maintaining class balance
train_files, val_files, test_files = [], [], []

for class_id, files in class_files.items():
    random.shuffle(files)

    num_samples = len(files)
    num_train = int(num_samples * 0.7)
    num_val = int(num_samples * 0.2)
    num_test = num_samples - (num_train + num_val)

    train_files.extend(files[:num_train])
    val_files.extend(files[num_train:num_train + num_val])
    test_files.extend(files[num_train + num_val:])

# ✅ Function to copy files
def copy_files(file_list, split):
    for label_file in file_list:
        image_file = label_file.replace(".txt", ".png")  # Assuming images are .jpg
        src_label = os.path.join(labels_dir, label_file)
        src_image = os.path.join(images_dir, image_file)

        dst_label = os.path.join(output_dirs[split]["labels"], label_file)
        dst_image = os.path.join(output_dirs[split]["images"], image_file)

        if not os.path.exists(src_image):
            print(f"⚠️ Image missing: {src_image}")
            continue  # Skip missing images

        shutil.copy(src_label, dst_label)
        shutil.copy(src_image, dst_image)

# ✅ Move files into respective folders
copy_files(train_files, "train")
copy_files(val_files, "val")
copy_files(test_files, "test")

print("✅ Dataset split complete with exact class ratios!")
print(f"Train: {len(train_files)} | Val: {len(val_files)} | Test: {len(test_files)}")
print(f"Files saved to: {output_base_dir}")


Found 1868 label files in D:\sonar seg\preprocessed\labels
✅ Dataset split complete with exact class ratios!
Train: 1302 | Val: 369 | Test: 197
Files saved to: D:\sonar seg\ttv


Training

In [6]:
import os
import numpy as np

# Path to label files
labels_dir = r"D:\sonar seg\preprocessed\labels"

# Number of classes (Update if necessary)
num_classes = 11  
class_counts = np.zeros(num_classes, dtype=int)

# Iterate over label files
for label_file in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_file), "r") as f:
        for line in f:
            class_id = int(line.split()[0])  # Extract class ID
            if 0 <= class_id < num_classes:
                class_counts[class_id] += 1

# Print class distribution
print("Class Distribution in Training Set:")
for i, count in enumerate(class_counts):
    print(f"Class {i}: {count} instances")


Class Distribution in Training Set:
Class 0: 468 instances
Class 1: 292 instances
Class 2: 320 instances
Class 3: 347 instances
Class 4: 171 instances
Class 5: 197 instances
Class 6: 111 instances
Class 7: 65 instances
Class 8: 615 instances
Class 9: 240 instances
Class 10: 1056 instances


In [7]:
import os

label_dir = r"D:\sonar seg\ttv\train\labels"
classes = set(range(11))  # Expected class IDs

for label_file in os.listdir(label_dir):
    with open(os.path.join(label_dir, label_file), "r") as f:
        for line in f:
            class_id = int(line.split()[0])
            if class_id not in classes:
                print(f"❌ Invalid class ID {class_id} in {label_file}")
print("✅ Label verification complete!")

✅ Label verification complete!


In [8]:
import os
import shutil
import random
from tqdm import tqdm
from ultralytics import YOLO
import yaml

In [10]:
import os

labels_dir = r"D:/sonar seg/ttv/train/labels"
empty_files = [f for f in os.listdir(labels_dir) if os.path.getsize(os.path.join(labels_dir, f)) == 0]

if empty_files:
    print(f" Warning: {len(empty_files)} empty label files found!")
    print(empty_files)

In [12]:
import os

image_dir = r"D:/sonar seg/ttv/train/images"
label_dir = r"D:/sonar seg/ttv/train/labels"

image_files = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')}
label_files = {os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.txt')}

missing_labels = image_files - label_files
missing_images = label_files - image_files

if missing_labels:
    print("🚨 Images without labels:", missing_labels)
if missing_images:
    print("🚨 Labels without images:", missing_images)

if not missing_labels and not missing_images:
    print(" All images have matching labels.")

 All images have matching labels.


In [13]:
empty_files = [f for f in os.listdir(label_dir) if os.path.getsize(os.path.join(label_dir, f)) == 0]

if empty_files:
    print(" Empty label files found:", empty_files)
else:
    print(" No empty label files.")

 No empty label files.


In [14]:
import torch
from ultralytics import YOLO

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" Using device: {device}")

# Define dataset YAML file path
DATA_YAML_PATH = r"D:\sonar seg\data.yaml"

# Define model checkpoint save path
OUTPUT_DIR = r"D:/sonar seg/ttv/runs/segment"

# Load YOLOv8 segmentation model
model = YOLO("yolov8n-seg.pt")  # Use 'yolov8m-seg.pt' or 'yolov8l-seg.pt' for better accuracy

# Train the model
model.train(
    data=DATA_YAML_PATH,  # Path to dataset config file
    epochs=20,            # Increase epochs for better results
    imgsz=640,            # Image size
    batch=16,             # Batch size
    device=device,        # Use GPU if available
    workers=4,            # Number of CPU threads
    project=OUTPUT_DIR,   # Where to save results
    name="yolov8_sonar_seg",  # Name of training run
    save=True,            # Save best model
    save_period=5,        # Save model every 5 epochs
    val=True,             # Run validation after training
    optimizer="AdamW"     # Optimizer: 'SGD' or 'AdamW' (AdamW recommended)
)

print(f" Training complete! Model saved in: {OUTPUT_DIR}")

 Using device: cuda
New https://pypi.org/project/ultralytics/8.3.88 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.83  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=D:\sonar seg\data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=4, project=D:/sonar seg/ttv/runs/segment, name=yolov8_sonar_seg3, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_n

train: Scanning D:\sonar seg\ttv\train\labels... 1302 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1302/1302 [00:02<00:00, 628.26it/s]


train: New cache created: D:\sonar seg\ttv\train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\sonar seg\ttv\val\labels.cache... 369 images, 0 backgrounds, 0 corrupt: 100%|██████████| 369/369 [00:00<?, ?it/s]


Plotting labels to D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg3\labels.jpg... 
optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg3
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20       4.5G     0.9045     0.5844     0.6147      1.084         15        640: 100%|██████████| 82/82 [00:51<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:09<00:00,  1.30it/s]

                   all        369        756      0.926      0.954      0.961      0.711      0.926      0.954      0.961      0.662



20 epochs completed in 0.624 hours.
Optimizer stripped from D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg3\weights\last.pt, 6.8MB
Optimizer stripped from D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg3\weights\best.pt, 6.8MB

Validating D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg3\weights\best.pt...
Ultralytics 8.3.83  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLOv8n-seg summary (fused): 85 layers, 3,260,209 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.77it/s]


                   all        369        756      0.926      0.954      0.961      0.711      0.926      0.954      0.961      0.661
                Bottle         76         85      0.893      0.986      0.987      0.772      0.893      0.986      0.987      0.694
                   Can         70         70      0.773      0.874      0.857      0.521      0.773      0.874      0.857      0.461
                 Chain         69         70      0.915      0.929      0.964       0.72      0.915      0.929      0.964      0.708
          Drink-carton         56         56       0.93      0.946      0.984      0.653       0.93      0.946      0.984      0.587
                  Hook         34         35      0.966      0.971      0.977      0.726      0.966      0.971      0.977      0.697
             Propeller         43         43      0.953      0.953       0.94      0.621      0.953      0.953       0.94      0.627
        Shampoo-bottle         21         21      0.975      0.952   

In [49]:
import torch
from ultralytics import YOLO

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" Using device: {device}")

# Define dataset YAML file path
DATA_YAML_PATH = r"D:\sonar seg\data.yaml"

# Define model checkpoint save path
OUTPUT_DIR = r"D:/sonar seg/ttv/runs/segment"

# Load the pre-trained YOLOv8 segmentation model (from the last checkpoint)
model = YOLO(r"D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg3\weights\last.pt")

# Continue training the model
model.train(
    data=DATA_YAML_PATH,  # Path to dataset config file
    epochs=20,            # Additional epochs for continued training
    imgsz=640,            # Image size
    batch=16,             # Batch size
    device=device,        # Use GPU if available
    workers=4,            # Number of CPU threads
    project=OUTPUT_DIR,   # Where to save results
    name="yolov8_sonar_seg_continued",  # New name for continued training run
    save=True,            # Save best model
    save_period=5,        # Save model every 5 epochs
    val=True,             # Run validation after training
    optimizer="AdamW"     # Optimizer: 'SGD' or 'AdamW' (AdamW recommended)
)

print(f" Training complete! Model saved in: {OUTPUT_DIR}")

 Using device: cuda
New https://pypi.org/project/ultralytics/8.3.88 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.83  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=segment, mode=train, model=D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg3\weights\last.pt, data=D:\sonar seg\data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=4, project=D:/sonar seg/ttv/runs/segment, name=yolov8_sonar_seg_continued, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, strea

train: Scanning D:\sonar seg\ttv\train\labels.cache... 1302 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1302/1302 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning D:\sonar seg\ttv\val\labels.cache... 369 images, 0 backgrounds, 0 corrupt: 100%|██████████| 369/369 [00:00<?, ?it/s]


Plotting labels to D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg_continued\labels.jpg... 
optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg_continued
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      2.51G     0.8457     0.5563     0.5391      1.049         15        640: 100%|██████████| 82/82 [00:29<00:00,  2.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:04<00:00,  2.72it/s]

                   all        369        756      0.932      0.947      0.956      0.722      0.933      0.947      0.957      0.657



20 epochs completed in 0.201 hours.
Optimizer stripped from D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg_continued\weights\last.pt, 6.8MB
Optimizer stripped from D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg_continued\weights\best.pt, 6.8MB

Validating D:\sonar seg\ttv\runs\segment\yolov8_sonar_seg_continued\weights\best.pt...
Ultralytics 8.3.83  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLOv8n-seg summary (fused): 85 layers, 3,260,209 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:05<00:00,  2.30it/s]


                   all        369        756      0.932      0.947      0.956      0.722      0.933      0.947      0.957      0.657
                Bottle         76         85      0.914      0.965      0.988      0.783      0.914      0.965      0.988       0.72
                   Can         70         70      0.836      0.799      0.873      0.561      0.836      0.799      0.873      0.482
                 Chain         69         70      0.907      0.929      0.971      0.723      0.893      0.914      0.962      0.708
          Drink-carton         56         56      0.943      0.964      0.982      0.653       0.96      0.982       0.99      0.554
                  Hook         34         35      0.953      0.971       0.98      0.753      0.953      0.971       0.98      0.711
             Propeller         43         43      0.946      0.953      0.931      0.645      0.946      0.953      0.931      0.609
        Shampoo-bottle         21         21      0.962      0.952   

Evaluation

In [48]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

# Paths to directories
TEST_IMAGES_DIR = r"D:\sonar seg\ttv\test\images"
TEST_LABELS_DIR = r"D:\sonar seg\ttv\test\labels"
PREDICTED_MASKS_DIR = r"D:\sonar seg\ttv\test\results\predict"

# Function to convert YOLO label to segmentation mask
def yolo_label_to_mask(label_path, image_shape):
    mask = np.zeros(image_shape, dtype=np.uint8)
    with open(label_path, 'r') as f:
        for line in f:
            parts = list(map(float, line.strip().split()))
            if len(parts) >= 5:  # Ensure there are at least 5 values (class_id, x_center, y_center, width, height)
                # Get the normalized coordinates (x_center, y_center, width, height)
                x_center, y_center, width, height = parts[1:5]  # Ignore additional values if any
                # Convert to pixel coordinates
                x_center = int(x_center * image_shape[1])
                y_center = int(y_center * image_shape[0])
                width = int(width * image_shape[1])
                height = int(height * image_shape[0])
                
                # Create a bounding box polygon (assuming rectangular masks)
                x1 = x_center - width // 2
                y1 = y_center - height // 2
                x2 = x_center + width // 2
                y2 = y_center + height // 2

                # Draw the polygon (bounding box) to the mask
                cv2.rectangle(mask, (x1, y1), (x2, y2), 1, -1)  # Assuming binary mask
    return mask

# Function to calculate pixel accuracy
def pixel_accuracy(true_mask, pred_mask):
    correct = np.sum(true_mask == pred_mask)
    total = true_mask.size
    return correct / total

# Function to calculate IoU (Intersection over Union)
def iou(true_mask, pred_mask):
    intersection = np.sum((true_mask == 1) & (pred_mask == 1))
    union = np.sum((true_mask == 1) | (pred_mask == 1))
    return intersection / union if union != 0 else 0

# Initialize lists for true and predicted masks
true_masks = []
pred_masks = []

# Iterate through the test set and calculate accuracy and IoU
for image_name in tqdm(os.listdir(TEST_IMAGES_DIR), desc="Evaluating model"):
    if image_name.endswith(('.jpg', '.png')):
        image_path = os.path.join(TEST_IMAGES_DIR, image_name)
        label_path = os.path.join(TEST_LABELS_DIR, image_name.replace(".jpg", ".txt").replace(".png", ".txt"))
        
        # Read the image
        image = cv2.imread(image_path)
        image_shape = image.shape[:2]  # Get image height and width

        # Generate the ground truth mask from YOLO label file
        true_mask = yolo_label_to_mask(label_path, image_shape)

        # Perform inference on the image
        results = model(image_path)  # Assuming model is loaded and ready to use

        # Extract the predicted mask (segmentation output)
        pred_mask = results[0].masks[0]  # This is likely a Masks object

        # Access the actual mask data from the Masks object
        pred_mask_data = pred_mask.data.cpu().numpy()  # If it is a PyTorch tensor, convert to NumPy array

        # Ensure pred_mask is binary (0 or 1) by thresholding
        pred_mask_data = (pred_mask_data > 0.5).astype(np.uint8)  # Threshold the mask to binary

        # If the mask has more than one channel, take the first channel
        if pred_mask_data.ndim == 3:
            pred_mask_data = pred_mask_data[:, :, 0]

        # Resize predicted mask to match true mask dimensions
        pred_mask_data_resized = cv2.resize(pred_mask_data, (true_mask.shape[1], true_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Append the masks to the lists
        true_masks.append(true_mask)
        pred_masks.append(pred_mask_data_resized)

# Calculate accuracy and IoU for the entire dataset
pixel_accuracies = [pixel_accuracy(true, pred) for true, pred in zip(true_masks, pred_masks)]
ious = [iou(true, pred) for true, pred in zip(true_masks, pred_masks)]

# Average accuracy and IoU
avg_pixel_accuracy = np.mean(pixel_accuracies)
avg_iou = np.mean(ious)

# Print the results
print(f"Average Pixel Accuracy: {avg_pixel_accuracy:.4f}")
print(f"Average IoU: {avg_iou:.4f}")

# Optionally, plot the confusion matrix
flat_true_masks = np.concatenate([mask.flatten() for mask in true_masks])
flat_pred_masks = np.concatenate([mask.flatten() for mask in pred_masks])

conf_matrix = confusion_matrix(flat_true_masks, flat_pred_masks)
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap="Blues", interpolation="nearest")
plt.title("Confusion Matrix")
plt.colorbar()
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

# Optionally, plot true vs predicted for an example
example_idx = 0  # Choose an index to visualize
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(true_masks[example_idx], cmap="gray")
plt.title("True Mask")

plt.subplot(1, 2, 2)
plt.imshow(pred_masks[example_idx], cmap="gray")
plt.title("Predicted Mask")

plt.show()

Evaluating model:   0%|          | 0/197 [00:00<?, ?it/s]


image 1/1 D:\sonar seg\ttv\test\images\marine-debris-aris3k-1003.png: 640x448 1 Valve, 55.0ms
Speed: 4.1ms preprocess, 55.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 448)


Evaluating model:   1%|          | 1/197 [00:00<00:49,  3.98it/s]


image 1/1 D:\sonar seg\ttv\test\images\marine-debris-aris3k-1021.png: 640x448 1 Can, 1 Wall, 53.4ms
Speed: 3.7ms preprocess, 53.4ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 D:\sonar seg\ttv\test\images\marine-debris-aris3k-1022.png: 640x448 1 Chain, 1 Drink-carton, 49.8ms
Speed: 3.8ms preprocess, 49.8ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 448)


Evaluating model:   2%|▏         | 3/197 [00:00<00:22,  8.60it/s]

...


image 1/1 D:\sonar seg\ttv\test\images\marine-debris-aris3k-990.png: 640x448 1 Chain, 1 Tire, 10.6ms
Speed: 1.4ms preprocess, 10.6ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 D:\sonar seg\ttv\test\images\marine-debris-aris3k-994.png: 640x448 1 Chain, 1 Drink-carton, 1 Hook, 1 Propeller, 1 Tire, 11.8ms
Speed: 1.6ms preprocess, 11.8ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 D:\sonar seg\ttv\test\images\marine-debris-aris3k-998.png: 640x448 1 Chain, 1 Wall, 11.5ms
Speed: 1.8ms preprocess, 11.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 448)


Evaluating model: 100%|██████████| 197/197 [00:05<00:00, 36.67it/s]

Average Pixel Accuracy: 0.6683
Average IoU: 0.0000


<Figure size 800x600 with 2 Axes>

<Figure size 1200x600 with 2 Axes>